In [15]:
#Fetching data from Wikipedia onto python combined list - tables
import requests
from bs4 import BeautifulSoup
import pandas as pd
    
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text, "html.parser")

#Creating tables and modifying it
tables = soup.find_all("table")[0].get_text().split('\n\n')
tables = [item for item in ','.join(tables).split('\nM') if item]

def extractDigits(lst): 
    return [''.join(el) for el in lst]

tables = extractDigits(tables)

print(tables)

[',Postal Code,Borough,Neighbourhood,', '1A,Not assigned,Not assigned,', '2A,Not assigned,Not assigned,', '3A,North York,Parkwoods,', '4A,North York,Victoria Village,', '5A,Downtown Toronto,Regent Park, Harbourfront,', '6A,North York,Lawrence Manor, Lawrence Heights,', "7A,Downtown Toronto,Queen's Park, Ontario Provincial Government,", '8A,Not assigned,Not assigned,', '9A,Etobicoke,Islington Avenue, Humber Valley Village,', '1B,Scarborough,Malvern, Rouge,', '2B,Not assigned,Not assigned,', '3B,North York,Don Mills,', '4B,East York,Parkview Hill, Woodbine Gardens,', '5B,Downtown Toronto,Garden District, Ryerson,', '6B,North York,Glencairn,', '7B,Not assigned,Not assigned,', '8B,Not assigned,Not assigned,', '9B,Etobicoke,West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale,', '1C,Scarborough,Rouge Hill, Port Union, Highland Creek,', '2C,Not assigned,Not assigned,', '3C,North York,Don Mills,', '4C,East York,Woodbine Heights,', '5C,Downtown Toronto,St. James Town,', '6C,Y

In [16]:
#Creating a Dataframe for this table
df_tables = pd.DataFrame(tables)
df_tables = df_tables[0].str.split(",",expand=True,)

df_tables.head()

,0,1,2,3,4,5,6,7,8,9,10
0,,Postal Code,Borough,Neighbourhood,,None,None,None,None,None,None
1,1A,Not assigned,Not assigned,,None,None,None,None,None,None,None
2,2A,Not assigned,Not assigned,,None,None,None,None,None,None,None
3,3A,North York,Parkwoods,,None,None,None,None,None,None,None
4,4A,North York,Victoria Village,,None,None,None,None,None,None,None


In [41]:
#Modifying this dataframe and storing it in csv

df_tables.rename(columns={0:'Postal Code',1:'Borough',2:'Neighbourhood'}, inplace = True)

df_tables.drop(df_tables.loc[df_tables['Postal Code']=='Postal Code'].index, inplace=True)
df_tables.drop(df_tables.loc[df_tables['Borough']=='Postal Code'].index, inplace=True)
df_tables.drop(df_tables.loc[df_tables['Borough']=='Not assigned'].index, inplace=True)

#pd.set_option('display.max_columns', 24)
#pd.set_option('display.max_rows', 103)

df_tables['Postal Code'] = 'M' + df_tables['Postal Code']

print(df_tables)

df_tables.fillna('', inplace=True)
#df_tables['Neighbourhood'] = df_tables['Neighbourhood'] + "," + df_tables[3] + "," + df_tables[4] + "," + df_tables[5] + "," + df_tables[6] + "," + df_tables[7] + "," + df_tables[8] + "," + df_tables[9] + "," + df_tables[10]

#df_tables.drop(df_tables.iloc[:, 3:], inplace = True, axis = 1)

df_tables['Neighbourhood1'] = df_tables['Neighbourhood'].str.strip(',')
df_tables.drop(columns="Neighbourhood", inplace = True)
df_tables.rename(columns={'Neighbourhood1':'Neighbourhood'}, inplace = True)

df_tables.to_csv('C:/Users/DELL/Desktop/Python_Programming/Toronto_Data.csv') 

#df_tables.drop(columns="level_0", inplace = True)
#df_tables.drop(columns="index", inplace = True)

df_tables


Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood, Latitude, Longitude]
Index: []


,Postal Code,Borough,Latitude,Longitude,Neighbourhood


In [40]:
df_tables

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [33]:
df_tables_2 = pd.read_csv (r'C:/Users/DELL/Desktop/Python_Programming/Geospatial_Coordinates.csv')
print (df_tables_2)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [20]:
df_tables.sort_values(by=['Postal Code'], inplace=True)
df_tables

KeyError: 'Postal Code'

In [34]:
df_tables = df_tables.merge(df_tables_2, on='Postal Code')
df_tables

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [22]:
options = ['East Toronto', 'West Toronto', 'Central Toronto', 'Downtown Toronto'] 

toronto_data = df_tables.loc[df_tables['Borough'].isin(options).reset_index(drop=True)]
toronto_data

KeyError: 'Borough'

In [23]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [24]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [25]:
import geocoder # import geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [26]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(latitude, longitude, toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_toronto

NameError: name 'toronto_data' is not defined